In [90]:
from tarfile import data_filter

import pandas as pd
import numpy as np
import importlib
from datetime import datetime
import matplotlib.pyplot as plt
from fontTools.misc.cython import returns
from matplotlib.pyplot import xticks
from pandas.core.interchange.dataframe_protocol import DataFrame
from ply.cpp import xrange
from pygments.lexer import bygroups
from sqlalchemy.dialects.mssql.information_schema import columns

import volpy_func_lib as vp
import load_clean_lib
import table_lib

from datetime import datetime
from volpy_func_lib import load_forward_price

In [110]:
importlib.reload(vp)
importlib.reload(load_clean_lib)
importlib.reload(table_lib)

<module 'table_lib' from 'D:\\Professionelt\\Code\\volpy\\table_lib.py'>

In [138]:
# Load summary_dly_df
date_str = "2025-03_15_14-10"
summary_dly_df = pd.read_csv(f"data/summary_dly_df {date_str}.csv")
summary_dly_df['date'] = pd.to_datetime(summary_dly_df['date'], format='%Y-%m-%d')
od_rdy = pd.read_csv(f"data/od_rdy {date_str}.csv")

In [140]:
RF = load_clean_lib.download_factor_df(Factor_list=["FF5"])[['date', 'RF']]
summary_dly_df.merge(RF[['date', 'RF']], on='date', how='left')

,Unnamed: 0,ticker,date,#days,low days,high days,low #K,high #K,#K,low SW,...,open,squared_return,SW_0_30,SW_m1_29,SW_0_29,SW_day,SW_day_return,SW_sell,SW_buy,RF
0,0,NDX,1996-01-04,5.0,16.0,44.0,30.0,23.0,26.5,0.095369,...,0.00,2.369752e-04,0.088424,NaN,0.088348,NaN,NaN,0.085411,NaN,0.00019
1,1,NDX,1996-01-05,5.0,15.0,43.0,29.0,27.0,28.0,0.083843,...,0.00,8.678916e-06,0.085796,0.088424,0.085817,-0.005468,-0.061839,0.082956,0.088424,0.00019
2,2,NDX,1996-01-08,5.0,12.0,40.0,27.0,28.0,27.5,0.093467,...,0.00,1.239744e-05,0.087939,0.085796,0.087891,-0.000835,-0.009727,0.084962,0.085796,0.00019
3,3,NDX,1996-01-09,5.0,11.0,39.0,26.0,25.0,25.5,0.155022,...,0.00,2.602734e-03,0.105957,0.087939,0.105566,0.014196,0.161427,0.102134,0.087939,0.00019
4,4,NDX,1996-01-10,5.0,10.0,38.0,28.0,29.0,28.5,0.144356,...,0.00,2.650384e-06,0.110652,0.105957,0.110407,0.000770,0.007270,0.106727,0.105957,0.00019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20884,20884,AMZN,2023-08-25,14.0,14.0,21.0,45.0,75.0,60.0,0.075159,...,132.47,1.160144e-04,0.089350,0.093322,0.089140,-0.007150,-0.076612,0.086173,0.093322,0.00020
20885,20885,AMZN,2023-08-28,14.0,11.0,18.0,42.0,66.0,54.0,0.075155,...,133.78,8.100000e-07,0.086167,0.089350,0.085914,-0.006300,-0.070507,0.083050,0.089350,0.00020
20886,20886,AMZN,2023-08-29,14.0,10.0,17.0,42.0,67.0,54.5,0.065194,...,133.38,1.767304e-04,0.081628,0.086167,0.081194,-0.007673,-0.089051,0.078494,0.086167,0.00020
20887,20887,AMZN,2023-08-30,14.0,9.0,16.0,41.0,64.0,52.5,0.059489,...,134.93,1.406596e-06,0.078171,0.081628,0.077607,-0.006608,-0.080948,0.075020,0.081628,0.00020


In [93]:
# Load factors
factor_df = pd.read_csv(f"data/factor_df.csv")

In [94]:
factor_df

,Unnamed: 0,date,Mkt,SMB,HML,RMW,CMA,UMD,BAB,QMJ,RF
0,0,1963-07-01,-0.0067,0.0002,-0.0035,0.0003,0.0013,-0.0021,0.004342,0.000507,0.00012
1,1,1963-07-02,0.0079,-0.0028,0.0028,-0.0008,-0.0021,0.0042,-0.001958,-0.000855,0.00012
2,2,1963-07-03,0.0063,-0.0018,-0.0010,0.0013,-0.0025,0.0041,-0.002134,0.001621,0.00012
3,3,1963-07-05,0.0040,0.0009,-0.0028,0.0007,-0.0030,0.0007,-0.001974,0.000064,0.00012
4,4,1963-07-08,-0.0063,0.0007,-0.0020,-0.0027,0.0006,-0.0045,0.004304,0.000070,0.00012
...,...,...,...,...,...,...,...,...,...,...,...
15476,15476,2024-12-24,0.0111,-0.0012,-0.0005,-0.0013,-0.0037,0.0067,NaN,NaN,0.00017
15477,15477,2024-12-26,0.0002,0.0109,-0.0019,-0.0044,0.0035,0.0001,NaN,NaN,0.00017
15478,15478,2024-12-27,-0.0117,-0.0044,0.0056,0.0041,0.0003,-0.0088,NaN,NaN,0.00017
15479,15479,2024-12-30,-0.0109,0.0024,0.0074,0.0055,0.0014,0.0006,NaN,NaN,0.00017


In [96]:
# summary_dly_df.merge(factor_df, on='date', how='left')

In [12]:
summary_dly_df.columns

Index(['Unnamed: 0', 'ticker', 'date', '#days', 'low days', 'high days',
       'low #K', 'high #K', '#K', 'low SW', 'high SW', 'Active',
       'Inactive reason', 'RV', 'open', 'squared_return', 'SW_0_30',
       'SW_m1_29', 'SW_0_29', 'SW_day', 'SW_day_return', 'SW_sell', 'SW_buy'],
      dtype='object')

In [70]:
factor_df_columns = ['Mkt', 'SMB', 'HML', 'RMW', 'CMA', 'UMD', 'BAB', 'QMJ', 'RF', 'SW_sell', 'SW_buy']
summary_df_column = ['SW_day']
other_columns = ['date']

keep_columns = factor_df_columns + summary_df_column + other_columns 

return_df = summary_dly_df.merge(factor_df, on='date', how='left')[keep_columns]
return_df = return_df.dropna()
return_df = return_df[["date"] + [col for col in return_df.columns if col != "date"]]
return_df['SW_day_rf'] = return_df['SW_sell'] - (1 + return_df["RF"]) * return_df["SW_buy"]
return_df = return_df.drop(columns={'SW_sell', 'SW_buy'})

In [72]:
def lm_regress(df, y_column, x_columns, print_summary = True):
    import statsmodels.api as sm
        
    # Define your target variable and factor variables
    X = df[x_columns]  # Independent variables
    y = -df[y_column]   # Dependent variable
    
    # Add a constant term to the independent variables for the intercept
    X = sm.add_constant(X)
    
    # Fit the linear regression model
    model = sm.OLS(y, X).fit()

    if print_summary:
        # Print the summary statistics
        print(model.summary())

    return model.summary()

In [73]:
target_column = 'SW_day'
feature_columns = ['Mkt', 'SMB', 'HML', 'RMW', 'CMA', 'UMD', 'BAB', 'QMJ', 'RF']

_ = lm_regress(return_df, y_column = target_column, x_columns = feature_columns)

                            OLS Regression Results                            
Dep. Variable:                 SW_day   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     216.9
Date:                Mon, 17 Mar 2025   Prob (F-statistic):               0.00
Time:                        10:57:14   Log-Likelihood:                 36175.
No. Observations:               20242   AIC:                        -7.233e+04
Df Residuals:                   20232   BIC:                        -7.225e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0021      0.000      5.305      0.0

In [74]:
target_column = 'SW_day_rf'
feature_columns = ['Mkt', 'SMB', 'HML', 'RMW', 'CMA', 'UMD', 'BAB', 'QMJ', 'RF']

_ = lm_regress(return_df, y_column = target_column, x_columns = feature_columns)

                            OLS Regression Results                            
Dep. Variable:              SW_day_rf   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     216.9
Date:                Mon, 17 Mar 2025   Prob (F-statistic):               0.00
Time:                        10:57:14   Log-Likelihood:                 36168.
No. Observations:               20242   AIC:                        -7.232e+04
Df Residuals:                   20232   BIC:                        -7.224e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0021      0.000      5.292      0.0

In [75]:
target_column = 'SW_day'
feature_columns = ['Mkt', 'RF']

_ = lm_regress(return_df, y_column = target_column, x_columns = feature_columns)

                            OLS Regression Results                            
Dep. Variable:                 SW_day   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.086
Method:                 Least Squares   F-statistic:                     956.8
Date:                Mon, 17 Mar 2025   Prob (F-statistic):               0.00
Time:                        10:58:13   Log-Likelihood:                 36157.
No. Observations:               20242   AIC:                        -7.231e+04
Df Residuals:                   20239   BIC:                        -7.228e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0022      0.000      5.632      0.0

In [76]:
target_column = 'SW_day_rf'
feature_columns = ['Mkt', 'RF']

_ = lm_regress(return_df, y_column = target_column, x_columns = feature_columns)

                            OLS Regression Results                            
Dep. Variable:              SW_day_rf   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.086
Method:                 Least Squares   F-statistic:                     956.8
Date:                Mon, 17 Mar 2025   Prob (F-statistic):               0.00
Time:                        10:58:13   Log-Likelihood:                 36150.
No. Observations:               20242   AIC:                        -7.229e+04
Df Residuals:                   20239   BIC:                        -7.227e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0022      0.000      5.618      0.0

In [77]:
target_column = 'SW_day'
feature_columns = ['RF']

_ = lm_regress(return_df, y_column = target_column, x_columns = feature_columns)

                            OLS Regression Results                            
Dep. Variable:                 SW_day   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     60.14
Date:                Mon, 17 Mar 2025   Prob (F-statistic):           9.27e-15
Time:                        10:59:01   Log-Likelihood:                 35272.
No. Observations:               20242   AIC:                        -7.054e+04
Df Residuals:                   20240   BIC:                        -7.053e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0027      0.000      6.588      0.0

In [79]:
target_column = 'SW_day_rf'
feature_columns = ['Mkt']

_ = lm_regress(return_df, y_column = target_column, x_columns = feature_columns)

                            OLS Regression Results                            
Dep. Variable:              SW_day_rf   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     1830.
Date:                Mon, 17 Mar 2025   Prob (F-statistic):               0.00
Time:                        10:59:20   Log-Likelihood:                 36112.
No. Observations:               20242   AIC:                        -7.222e+04
Df Residuals:                   20240   BIC:                        -7.220e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0047      0.000     16.328      0.0

In [80]:
target_column = 'SW_day_rf'
feature_columns = ['Mkt', 'SMB', 'HML', 'RMW', 'CMA', 'UMD', 'BAB', 'QMJ', 'RF']

_ = lm_regress(return_df, y_column = target_column, x_columns = feature_columns)

                            OLS Regression Results                            
Dep. Variable:              SW_day_rf   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     216.9
Date:                Mon, 17 Mar 2025   Prob (F-statistic):               0.00
Time:                        10:59:39   Log-Likelihood:                 36168.
No. Observations:               20242   AIC:                        -7.232e+04
Df Residuals:                   20232   BIC:                        -7.224e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0021      0.000      5.292      0.0

In [83]:
target_column = 'SW_day_rf'
feature_columns = ['Mkt','BAB']

_ = lm_regress(return_df, y_column = target_column, x_columns = feature_columns)

                            OLS Regression Results                            
Dep. Variable:              SW_day_rf   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     924.9
Date:                Mon, 17 Mar 2025   Prob (F-statistic):               0.00
Time:                        11:01:03   Log-Likelihood:                 36121.
No. Observations:               20242   AIC:                        -7.224e+04
Df Residuals:                   20239   BIC:                        -7.221e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0046      0.000     16.080      0.0

In [88]:
target_column = 'SW_day_rf'
feature_columns = ['Mkt', 'SMB', 'HML', 'RMW', 'CMA']

_ = lm_regress(return_df, y_column = target_column, x_columns = feature_columns)

                            OLS Regression Results                            
Dep. Variable:              SW_day_rf   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     367.7
Date:                Mon, 17 Mar 2025   Prob (F-statistic):               0.00
Time:                        11:02:34   Log-Likelihood:                 36116.
No. Observations:               20242   AIC:                        -7.222e+04
Df Residuals:                   20236   BIC:                        -7.217e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0047      0.000     16.321      0.0

In [89]:
target_column = 'SW_day_rf'
feature_columns = ['Mkt', 'SMB', 'HML']

_ = lm_regress(return_df, y_column = target_column, x_columns = feature_columns)

                            OLS Regression Results                            
Dep. Variable:              SW_day_rf   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     612.5
Date:                Mon, 17 Mar 2025   Prob (F-statistic):               0.00
Time:                        11:02:43   Log-Likelihood:                 36115.
No. Observations:               20242   AIC:                        -7.222e+04
Df Residuals:                   20238   BIC:                        -7.219e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0047      0.000     16.349      0.0